***

### Logic
#### Backlog giao: Bưu cục nhận hàng giao từ tải => Giao thành công/Trả hàng
    * Không quá 120h.
#### Backlog trả: Bưu cục nhận hàng trả từ tải => Trả hàng thành công
    * Không quá 72h.
#### Backlog lấy
    * Không quá 72h?
#### Backlog luân chuyển lấy: Lấy hàng thành công => Bàn giao tải 
    * HCM -> HN: 12h.
    * Các tỉnh còn lại: 32h.
#### Backlog luân chuyển trả: Chuyển trạng thái trả hàng => Bàn giao tải
    * Giao hàng tại HCM HN: 36h.
    * Giao hàng tại các tỉnh còn lại: 56h.
***

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import os


# change current working directory to Downloads
os.chdir('/Users/phuoc/Downloads')

# read data
export = pd.read_excel('export.xlsx')
inside = pd.read_excel('inside.xlsx')

# old_name: new_name -> replace inside headers
renamed_inside_headers = {
    'Mã đơn': 'MaDH',
    'Mã kiện': 'MaKien',
    'Kho gửi': 'KhoGui',
    'Kho nhận': 'KhoNhan',
    'Kho hiện tại': 'KhoHienTai',
    'TG đóng kiện': 'TGDongKien',
    'TG cập nhật': 'TGCapNhat',
    'TG nhận kiện': 'TGNhanKien',
    'TG kết thúc': 'TGKetThuc',
    'Trạng thái': 'TrangThaiLuanChuyen',
    'Số đơn': 'SoDon',
    'Khối lượng': 'KhoiLuong',
    'Mã niêm phong đóng': 'MaNiemPhongDong',
    'Mã niêm phong nhận': 'MaNiemPhongNhan',
    'Hình thức đóng gói': 'HinhThucDongGoi',
    'Hình thức vận chuyển': 'HinhThucVanChuyen',
    'Ghi chú': 'GhiChu'
}

# rename columns name of inside dataframe
inside = inside.rename(columns=renamed_inside_headers)

# normalize datetime of inside dataframe
# inside['TGDongKien'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S'))

# change str to datetime
inside[[
    'TGDongKien',
    'TGCapNhat',
    'TGNhanKien',
    'TGKetThuc'
]] = inside[[
    'TGDongKien',
    'TGCapNhat',
    'TGNhanKien',
    'TGKetThuc'
]].apply(lambda x: pd.to_datetime(x, format='%d/%m/%Y %H:%M:%S'))
inside['KhoGui'] = inside['KhoGui'].astype(int)
inside['KhoNhan'] = inside['KhoNhan'].astype(int)
inside['KhoHienTai'] = inside['KhoHienTai'].astype(int)

# normalize datetime of export dataframe
# replace all <nil> values
export = export.replace('<nil>', np.nan)

# change str to datetime
export[[
    'ThoiGianTao',
    'ThoiGianTaoChuyenDoi',
    'ThoiGianKetThucLay',
    'ThoiGianGiaoLanDau',
    'ThoiGianKetThucGiao',
    'ThoiGianGiaoHangMongMuon',
    'TGKetThucTra',
]] = export[[
        'ThoiGianTao',
        'ThoiGianTaoChuyenDoi',
        'ThoiGianKetThucLay',
        'ThoiGianGiaoLanDau',
        'ThoiGianKetThucGiao',
        'ThoiGianGiaoHangMongMuon',
        'TGKetThucTra',
]].apply(pd.to_datetime)

# set display of dataframe
pd.set_option('display.max_columns', 88)
pd.set_option('display.max_rows', 100)

# join export with inside
data = pd.merge(export, inside[['MaDH','MaKien', 'KhoGui', 'KhoNhan', 'TGNhanKien', 'TrangThaiLuanChuyen']], on='MaDH', how='left')

# codes of e-commerces exchange
shopee_codes = ['18692']
sendo_codes = ['1539', '1160902', '1160904', '1160905']
tiki_codes = ['1367']
lazada_codes = ['1041351', '9794']
ecoms = shopee_codes + sendo_codes + tiki_codes + lazada_codes

# classify types of backlog
# delivery backlog (backlog giao)
delivery_filter = data['TrangThai'].isin([
    'Đang giao hàng', 'Giao hàng không thành công'
]) | (((data['TrangThai'] == 'Lưu kho') | (data['TrangThai'] == 'Lấy hàng thành công') | (data['TrangThai'] == 'Đang trung chuyển hàng')) &\
      (data['KhoGiao'] == data['KhoHienTai'])) | ((data['TrangThai'] == 'Chờ giao lại') & (datetime.now() - data['ThoiGianKetThucGiao'] > pd.Timedelta('24 hours')))

delivery = data.loc[delivery_filter]
delivery['LoaiBacklog'] = 'Kho giao'
internal_delivery_filter = delivery['KhoLay'] == delivery['KhoHienTai']
delivery.loc[internal_delivery_filter, 'N0'] = delivery.loc[internal_delivery_filter]['ThoiGianKetThucLay']
delivery.loc[~internal_delivery_filter, 'N0'] = delivery.loc[~internal_delivery_filter]['TGNhanKien']
delivery['N+'] = delivery['N0'] + pd.Timedelta(hours=120)
delivery['Aging'] = (datetime.today() - delivery['N+']).fillna(pd.Timedelta(hours=9999))


# return backlog (backlog trả)
returned_filter = data['TrangThai'].isin([
    'Đang hoàn hàng', 'Hoàn hàng không thành công'
    ]) | (((data['TrangThai'] == 'Chuyển hoàn') | (data['TrangThai'] == 'Đang trung chuyển hàng hoàn')) & (((~data['KhoTra'].isnull()) & (data['KhoTra'] == data['KhoHienTai'])) |\
                                                 ((data['KhoTra'].isnull()) & (data['KhoLay'] == data['KhoHienTai']))))
returned = data.loc[returned_filter]
returned['LoaiBacklog'] = 'Kho trả'
internal_return_filter = returned['KhoHienTai'] == returned['KhoGiao']
returned.loc[internal_return_filter, 'N0'] = returned.loc[internal_return_filter]['ThoiGianKetThucGiao']
returned.loc[~internal_return_filter, 'N0'] = returned.loc[~internal_return_filter]['TGNhanKien']
returned['N+'] = returned['N0'] + pd.Timedelta(hours=72)
returned['Aging'] = (datetime.today() - returned['N+']).fillna(pd.Timedelta(hours=9999))


# pickup backlog (backlog lấy)
pickup_filter = (data['TrangThai'].isin(['Chờ lấy hàng', 'Đang lấy hàng', 'Lấy không thành công'])) |\
              ((data['TrangThai'] == 'Tạo thành công') & (data['ThoiGianKetThucLay'].isnull()) &\
               (data['KhoHienTai'] == data['KhoLay']))
pickup = data.loc[pickup_filter]
pickup['LoaiBacklog'] = 'Kho lấy'
ecoms_filter = pickup['MaKH'].isin(ecoms)
pickup.loc[ecoms_filter, 'N0'] = pickup.loc[ecoms_filter]['ThoiGianTaoChuyenDoi']
pickup.loc[~ecoms_filter, 'N0'] = pickup.loc[~ecoms_filter]['ThoiGianTao']
pickup['N+'] = pickup['N0'] + pd.Timedelta(hours=72)
pickup['Aging'] = (datetime.today() - pickup['N+']).fillna(pd.Timedelta(hours=9999))

# transporting backlog (backlog luân chuyển)
trans_filter = (((data['TrangThai'] == 'Lấy hàng thành công') | (data['TrangThai'] == 'Lưu kho')) & (data['KhoGiao'] != data['KhoHienTai'])) |\
                       ((data['TrangThai'] == 'Đang trung chuyển hàng') & (data['KhoLay'] != data['KhoGiao']))
transporting = data.loc[trans_filter]
transporting['LoaiBacklog'] = 'Kho lấy luân chuyển'
transporting['N0'] = transporting['ThoiGianKetThucLay']
transporting['N+'] = transporting['N0']+ pd.Timedelta(hours=12) # Kho lay tai HCM or HN
# transporting['N+'] = transporting['N0'] + pd.Timedelta(hours=32) # Kho lay tai cac tinh con lai
transporting['Aging'] = (datetime.today() - transporting['N+']).fillna(pd.Timedelta(hours=9999))


# returned_transporting backlog (backlog luân chuyển trả hàng)
rt_filter = ((data['TrangThai'].isin(['Chuyển hoàn', 'Chờ chuyển hoàn']) | (data['TrangThai'] == 'Đang trung chuyển hàng hoàn')) &\
             ((data['KhoTra'].isnull() & (data['KhoLay'] != data['KhoHienTai'])) |
              (~data['KhoTra'].isnull() & (data['KhoTra'] != data['KhoHienTai']))))
r_transporting = data.loc[rt_filter]
r_transporting['LoaiBacklog'] = 'Kho giao luân chuyển'
r_transporting['N0'] = r_transporting['ThoiGianKetThucGiao']
r_transporting['N+'] = r_transporting['N0'] + pd.Timedelta(hours=36) # Kho giao tai HCM or HN
# r_transporting['N+'] = r_transporting['N0'] + pd.Timedelta(hours=56) # Kho giao tai cac tinh con lai
r_transporting['Aging'] = (datetime.today() - r_transporting['N+']).fillna(pd.Timedelta(hours=9999))


# composite data
data = delivery.append([returned, pickup, transporting, r_transporting])
# set e-commerces column to NaN
data['ECommerces'] = np.nan


# name of e-commerces exchange
# shopee
data.loc[data['MaKH'].isin(shopee_codes), 'ECommerces'] = 'Shopee'
# sendo
data.loc[data['MaKH'].isin(sendo_codes), 'ECommerces'] = 'Sendo'
# tiki
data.loc[data['MaKH'].isin(tiki_codes), 'ECommerces'] = 'Tiki'
# lazada
data.loc[data['MaKH'].isin(lazada_codes), 'ECommerces'] = 'Lazada'

# ToanTrinh
data['Aging_ToanTrinh'] = np.nan
ecoms_filter = data['MaKH'].isin(ecoms)
data.loc[ecoms_filter, 'Aging_ToanTrinh'] = datetime.today() - data[ecoms_filter]['ThoiGianTaoChuyenDoi']
data.loc[~ecoms_filter, 'Aging_ToanTrinh'] = datetime.today() - data[~ecoms_filter]['ThoiGianTao']
# reset the index of datafram
data.reset_index(inplace=True)
del data['index']
data['KhoGui'].astype('int64', errors='ignore')
data['NgayHienTai'] = datetime.today().strftime('%Y-%m-%d')
data
# data['KhoGui']

,MaDH,MaDHShop,ThoiGianTao,ThoiGianTaoChuyenDoi,ThoiGianKetThucLay,ThoiGianGiaoLanDau,ThoiGianKetThucGiao,ThoiGianGiaoHangMongMuon,TGKetThucTra,ThongTinTraHang,TuQuan,TuTinh,TuVung,DenQuan,DenTinh,DenVung,KhoLay,KhoGiao,KhoHienTai,KhoTra,TrangThai,TrongLuong,KichThuoc,MaKH,GhiChuGHN,GhiChu,GoiCuoc,SoLanLay,SoLanGiao,SoLanTra,TinhTrangLuanChuyen,MaNVLay,MaNVGiao,TenNguoiGui,MaDiaChiLay,MaKien,KhoGui,KhoNhan,TGNhanKien,TrangThaiLuanChuyen,LoaiBacklog,N0,N+,Aging,ECommerces,Aging_ToanTrinh,NgayHienTai
0,8FHUSF65Y7,SAPO295733036,2020-08-30 20:01:43,2020-08-31 08:00:00,2020-08-31 14:30:08,2020-09-08 10:05:20,NaT,2020-09-07 15:30:00,NaT,NaN,Huyện Tiên Du,Bắc Ninh,B,Huyện Nhà Bè,Hồ Chí Minh,E,Bưu Cục Tiên Du-Bắc Ninh,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục Tiên Du-Bắc Ninh,Lưu kho,150,10x10x10,2543339,'08/09/2020 10:06:15 - GHN-DFC1A0|THAY ĐỔI THÔ...,"'Gọi lại trước khi hoàn, khách cần gấp. Hàng d...",3 Ngày,1,1,0,NaN,1706771,NaN,'Bé Bồng',1287025,20REX63455,1334.0,2275.0,2020-09-07 23:03:53,Kết thúc,Kho giao,2020-09-07 23:03:53,2020-09-12 23:03:53,-4 days +00:46:31.399422,NaN,9 days 03:48:41.634825,2020-09-08
1,GHNST0000330650VN,GHNST0000330650VN,2020-08-30 21:44:20,2020-08-31 08:00:00,2020-09-03 22:28:44,2020-09-05 08:25:20,NaT,2020-09-08 23:00:00,NaT,NaN,Quận Long Biên,Hà Nội,G,Quận 7,Hồ Chí Minh,E,Kho Khách hàng lớn Hà Nội,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 76 Bùi Thị Cội-Dĩ An-Bình Dương,Lưu kho,160,12x6x2,9794,'05/09/2020 09:10:31 - GHN-DFC1A1|Khách đổi đị...,NaN,2 Ngày,NaN,1,0,0,255462,NaN,'Jizetin Flagship Store',2502649,20EVW92020,2329.0,2275.0,2020-09-08 14:14:11,Kết thúc,Kho giao,2020-09-08 14:14:11,2020-09-13 14:14:11,-5 days +09:36:13.399422,Lazada,8 days 15:50:24.584104,2020-09-08
2,8FHDQR1RY9,200830062555,2020-08-30 18:03:37,2020-08-31 08:00:00,2020-08-30 20:34:50,2020-09-08 12:50:14,NaT,2020-09-04 23:00:00,NaT,NaN,Quận Cầu Giấy,Hà Nội,G,Huyện Nhà Bè,Hồ Chí Minh,E,Kho Khách hàng lớn Hà Nội,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Kho Khách hàng lớn Hà Nội,Lưu kho,2000,10x10x10,1156533,'08/09/2020 05:52:39 - GHN-DFC1A0|THAY ĐỔI THÔ...,'ISHTT340612',2 Ngày,1,1,0,NaN,1806445,NaN,'iShoesvn',1028477,20WQG48773,1334.0,2275.0,2020-09-07 23:02:08,Kết thúc,Kho giao,2020-09-07 23:02:08,2020-09-12 23:02:08,-4 days +00:48:16.399422,NaN,9 days 05:46:47.634825,2020-09-08
3,8FH14SY457,2427469664432128627,2020-08-30 08:04:38,2020-08-31 08:00:00,2020-08-31 16:43:42,2020-09-08 17:11:16,NaT,2020-09-04 17:54:00,NaT,NaN,Quận 9,Hồ Chí Minh,E,Huyện Nhà Bè,Hồ Chí Minh,E,Bưu Cục 05 Đường 8-Q.9-HCM,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 05 Đường 8-Q.9-HCM,Lưu kho,1050,20x30x5,18692,'08/09/2020 11:45:08 - GHN-DFC1A0|THAY ĐỔI THÔ...,'Không cho xem hàng. Khối lượng nhận hàng tối ...,1 Ngày,1,1,0,NaN,1881630,NaN,'Nông Sản Sạch Nhà Trồng',1154025,20ZXT58773,1334.0,2275.0,2020-09-07 23:04:19,Kết thúc,Kho giao,2020-09-07 23:04:19,2020-09-12 23:04:19,-4 days +00:46:05.399422,Shopee,8 days 15:50:24.584104,2020-09-08
4,8FHR7S314Y,2427679379632128260,2020-08-30 15:01:18,2020-08-31 08:00:00,2020-08-31 09:47:37,2020-09-08 12:42:13,NaT,2020-09-07 15:30:00,NaT,NaN,Huyện Xuân Trường,Nam Định,C,Huyện Nhà Bè,Hồ Chí Minh,E,Bưu Cục Trực Ninh-Nam Định,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục Trực Ninh-Nam Định,Lưu kho,500,10x10x10,18692,'08/09/2020 05:53:55 - GHN-DFC1A2|KHÔNG LIÊN L...,'Không cho xem hàng. Khối lượng nhận hàng tối ...,3 Ngày,1,1,0,NaN,1868082,NaN,'Kem_Authentic',1154025,20ZPD14912,1334.0,2275.0,2020-09-07 23:03:21,Kết thúc,Kho giao,2020-09-07 23:03:21,2020-09-12 23:03:21,-4 days +00:47:03.399422,Shopee,8 days 15:50:24.584104,2020-09-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5048,FI69FFAU,SKVU04KTR - 1599120548.44874,2020-09-03 15:09:08

In [3]:
# export data backlog to csv file
# data export
data_to_export = data[[
    'NgayHienTai',
    'MaDH',
    'KhoHienTai',
    'TrangThai',
    'GhiChuGHN',
    'SoLanLay',
    'SoLanGiao',
    'SoLanTra',
    'LoaiBacklog',
    'N0',
    'N+',
    'Aging',
    'Aging_ToanTrinh',
    'ECommerces',
    'TrangThaiLuanChuyen',
]]

data_to_export.to_csv('backlog.csv', encoding='utf-8-sig', index=False)


In [4]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as df2g


# define scope of google api json keyfile
SCOPES = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]

# credentials to google service account
credentials = ServiceAccountCredentials.from_json_keyfile_name('ggapi_key.json', scopes=SCOPES)
# authorize with gspread
gc = gspread.authorize(credentials)

SPREADSHEET_ID = '1_UlAs1Ed6rxl1TNM8Vv2fnEsRT61BOWTVzCMtd1hwME'  # id of sheets (on url)
WKS_NAME = 'Backlog_bot'
active_sheets = gc.open_by_key(SPREADSHEET_ID)
wks = active_sheets.worksheet(WKS_NAME)
data_upload = data_to_export.loc[data_to_export['Aging'] >= pd.Timedelta(hours=0)]
# update data to google sheets
wks.clear()
df2g.upload(
    data_upload,
    SPREADSHEET_ID,
    WKS_NAME,
    credentials=credentials,
    row_names=False,
)


<Worksheet 'Backlog_bot' id:181055436>

In [5]:
# read dataframe from google sheet
from df2gspread import gspread2df as g2df

df_gspread = g2df.download(gfile=SPREADSHEET_ID, wks_name=WKS_NAME, credentials=credentials, col_names=True, row_names=False)

# df_gspread

In [21]:
# Inventory report
# from df2gspread import df2gspread as df2g
data_inventory = data.loc[(data['LoaiBacklog'] == 'Kho giao') & (data['TrangThai'] != 'Đang giao hàng')]
data_inventory['LoaiXuLy'] = 'Chưa giao lại'
chuagiao_filter = data_inventory['SoLanGiao'].isnull()
giaoloi_filter = ~data_inventory['GhiChuGHN'].isnull() & data_inventory['GhiChuGHN'].str.contains(datetime.today().strftime('%d/%m/%Y'))
data_inventory.loc[chuagiao_filter, 'LoaiXuLy'] = 'Chưa giao lần nào'
data_inventory.loc[giaoloi_filter, 'LoaiXuLy'] = 'Giao lỗi'

def convert_to_day(x):
    try:
        return x.strftime('%Y-%m-%d')
    except ValueError:
        return np.nan

data_inventory['N_ve_kho'] = data_inventory['N0'].apply(lambda x: convert_to_day(x))

del data_inventory['N0']
del data_inventory['N+']
del data_inventory['Aging']
del data_inventory['NgayHienTai']
del data_inventory['GhiChu']
# data_inventory
# to csv file
data_inventory.to_csv('rp_giao.csv', index=False, encoding='utf-8-sig')
# data_inventory.to_excel('report_giao.xlsx')

# pivot data and push to google sheet
table1 = pd.pivot_table(data_inventory, values='MaDH', index=['LoaiXuLy'], aggfunc='count')
table2 = pd.pivot_table(data_inventory, values='MaDH', index=['LoaiXuLy'], columns=['N_ve_kho'], aggfunc='count', fill_value=0)

# table1
WKS_INVENTORY = 'RPGiao_bot'
# inventory_wks = gc.open_by_key(SPREADSHEET_ID).worksheet(WKS_INVENTORY)
# # inventory_wks.clear()  # clear old data
df2g.upload(
    table1,
    SPREADSHEET_ID,
    WKS_INVENTORY,
    credentials=credentials,
    row_names=True,
    col_names=True,
    start_cell='A1',
)

# inventory_wks.add_row(table2.shape[0])

df2g.upload(
    table2,
    SPREADSHEET_ID,
    WKS_INVENTORY,
    credentials=credentials,
    row_names=True,
    col_names=True,
    clean=False,
    start_cell='A7',
)
# then write detail inventory data
inventory_upload = data_inventory[[
    'MaDH',
    'KhoHienTai',
    'TrangThai',
    'GhiChuGHN',
    'SoLanGiao',
    'ECommerces',
    'LoaiXuLy',
    'N_ve_kho',
    'TrangThaiLuanChuyen',
    'MaKien',
]]
df2g.upload(
    inventory_upload,
    SPREADSHEET_ID,
    WKS_INVENTORY,
    credentials=credentials,
    row_names=False,
    col_names=True,
    clean=False,
    start_cell='A12',
)

<Worksheet 'RPGiao_bot' id:1115912348>

In [15]:
# from datetime import datetime

# dt = datetime.strptime('08/08/2020 14:10:12', '%d/%m/%Y %H:%M:%S')
# print(dt)
# def iter_df(df):
#     """Generator for df."""
#     for val in df.columns:
#         yield val
#     for row in df.to_numpy():
#         for val in row:
#             if pd.isna(val):
#                 yield ''
#             else:
#                 yield val